In [1]:
import pyvista as pv
import numpy as np
import os, flopy
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation, LinearTriInterpolator
from tqdm import tqdm

    
name = 'Model'
workspace = '../model'
mf_exe_name = '../exe/mf6'

sim = flopy.mf6.MFSimulation.load(sim_name=name, exe_name=mf_exe_name, sim_ws=workspace)

mfmodel = sim.get_model(model_name='model')

fname = os.path.join(workspace, name + '.hds')
hdobj = flopy.utils.HeadFile(fname, precision='double')
head = hdobj.get_data()
headRshp = head.reshape((head.shape[0],head.shape[2]))

flopy is installed in /home/gida2/.local/lib/python3.6/site-packages/flopy
loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package disv...
    loading package npf...
    loading package ic...
    loading package rch...
    loading package evt...
    loading package drn...
    loading package oc...
  loading ims package model...


In [2]:

mTopCent = mfmodel.disv.top.get_data()
verticesXYArray = np.dstack((mfmodel.disv.vertices.array.xv,mfmodel.disv.vertices.array.yv))[0]
centroidXYArray = np.dstack((mfmodel.disv.cell2d.array.xc,mfmodel.disv.cell2d.array.yc))[0]

vertexIndexList = []
for index, cell in enumerate(mfmodel.disv.cell2d.array):
    vertexIndexList.append([x for x in list(cell)[4:] if x is not None])

def triInterpolation(zValue, xPoints, yPoints):
    triObj = Triangulation(xPoints, yPoints)
    fz = LinearTriInterpolator(triObj, zValue)
    return fz
def interpVertices(zValues, xyCentroid, xyVertices):
    interpVerticesList=[]
    #print(xyArray.shape)
    layTri = triInterpolation(zValues, xyCentroid[:,0], xyCentroid[:,1])
    
    for index, vertice in tqdm(enumerate(xyVertices)):
        newZ = layTri(vertice[0],vertice[1])
        #if it is a 
        if float(newZ)!=float(newZ):
            for index2, row2 in enumerate(vertexIndexList):
                #print(row2)
                if index in row2:
                    #print(zValues[index2])
                    newZ = zValues[index2]
        interpVerticesList.append(newZ)
    interpVerticesArray = np.array(interpVerticesList)
    return interpVerticesArray

In [3]:
nlay = mfmodel.disv.nlay.array
nvert = mfmodel.disv.nvert.array
ncpl = mfmodel.disv.ncpl.array
modelCellZ = np.vstack((mfmodel.disv.top.array,mfmodel.disv.botm.array))
modelVertZ = np.zeros((nlay+1,mfmodel.disv.nvert.array))
#print(modelVertZ.shape)
for i in range(modelCellZ.shape[0]):
    modelVertZ[i] = interpVertices(modelCellZ[i], centroidXYArray, verticesXYArray)

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Warning: converting a masked element to nan.
99154it [00:08, 12152.30it/s]
99154it [00:08, 11332.93it/s]
99154it [00:08, 11150.71it/s]
99154it [00:08, 11621.70it/s]
99154it [00:08, 11802.81it/s]
99154it [00:08, 11844.01it/s]
99154it [00:08, 11521.26it/s]


### for model geometry

In [4]:

cellTubes = {}
for index, cell in tqdm(enumerate(mfmodel.disv.cell2d.array)):
    vertexIndexList = [x for x in list(cell)[4:] if x is not None]
    filterVertexIndexList = [len(vertexIndexList)] + list(range(len(vertexIndexList)))
    filterVertexIndexArray = np.array(filterVertexIndexList)
    #create polydata for lay
    layerTubes = {}
    for lay in range(nlay):
        filterVertexArray = np.array([list(verticesXYArray[vertex]) + [modelVertZ[lay,vertex]] for vertex in vertexIndexList])
        cellSurf = pv.PolyData(filterVertexArray, filterVertexIndexArray)
        #cellSurf.cell_arrays['head'] = head[lay][0][index]
        cellZ = modelCellZ[:,index]
        cellMesh = cellSurf.extrude([0, 0, cellZ[lay+1]-cellZ[lay]])
        layerTubes[str(lay)] = cellMesh  
    blocks = pv.MultiBlock(layerTubes)
    cellTubes[str(index)] = blocks.combine()
totalModelCells = pv.MultiBlock(cellTubes)
totalModelMerged = totalModelCells.combine()
totalModelMerged.save('../vtk/modelGeometry.vtk')

22275it [01:00, 367.87it/s]


KeyboardInterrupt: 

### for cell heads

### for water table

### for drain boundary

In [5]:

drnPkg = mfmodel.get_package('drn_0')
drainTubes = {}
for index, drnCell in tqdm(enumerate(drnPkg.stress_period_data.array[0])):
    cellNumber = drnCell[0][1]
    cellLay = drnCell[0][0]
    cellGrid = mfmodel.disv.cell2d.array[cellNumber]
    vertexIndexList = [x for x in list(cellGrid)[4:] if x is not None]
    filterVertexIndexList = [len(vertexIndexList)] + list(range(len(vertexIndexList)))
    filterVertexIndexArray = np.array(filterVertexIndexList)
    filterVertexArray = np.array([list(verticesXYArray[vertex]) + [modelCellZ[cellLay,cellNumber]] for vertex in vertexIndexList])
    cellSurf = pv.PolyData(filterVertexArray, filterVertexIndexArray)
    cellSurf.cell_arrays['elevation'] = drnCell[1]
    cellSurf.cell_arrays['cond'] = drnCell[2]
    cellZ = modelCellZ[:,cellNumber]
    cellMesh = cellSurf.extrude([0, 0, cellZ[cellLay +1]-cellZ[cellLay]])
    drainTubes[str(index)] = cellMesh 
totalDrainsCells = pv.MultiBlock(drainTubes)
totalDrainsMerged = totalDrainsCells.combine()
totalDrainsMerged.save('../vtk/modelDrains.vtk')

10185it [00:06, 1694.00it/s]


In [ ]:
import pyvista as pv
import numpy as np
import os, flopy
import matplotlib.pyplot as plt
name = 'Model'
workspace = '../model'
mf_exe_name = '../exe/mf6'

sim = flopy.mf6.MFSimulation.load(sim_name=name, exe_name=mf_exe_name, sim_ws=workspace)

mfmodel = sim.get_model(model_name='model')

fname = os.path.join(workspace, name + '.hds')
hdobj = flopy.utils.HeadFile(fname, precision='double')
head = hdobj.get_data()
headRshp = head.reshape((head.shape[0],head.shape[2]))

mtop = np.ones([mfmodel.disv.nvert.array ])*mfmodel.disv.top[0]
verticesXYArray = np.dstack((mfmodel.disv.vertices.array.xv,mfmodel.disv.vertices.array.yv))[0]

In [ ]:
modelz = np.vstack((mfmodel.disv.top.array,mfmodel.disv.botm.array))
nlay = mfmodel.disv.nlay.array

In [ ]:
cellSurf = pv.PolyData()

In [ ]:
mtop